# Backtesting Trading Strategies with Seaborn

---

This notebook demonstrates how to implement and evaluate simple trading strategies using historical stock data.  
We will use **pandas** for data manipulation, **seaborn** for visualization, and **matplotlib** for plotting.

---

### Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import talib
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

##  Strategy Implementations

---

### 1. Simple Moving Average (SMA)
Buys when the short-term average crosses up through the long-term average (golden cross). Sells when the short-term average crosses down through the long-term average (death cross).

In [260]:
class SMAcrossover(Strategy):
    display_name = "SMAcrossover"
    short = 21
    long = 50

    def init(self):
        close = self.data.Close
        self.sma_short = self.I(talib.SMA, close, self.short)
        self.sma_long = self.I(talib.SMA, close, self.long)

    def next(self):
        if crossover(self.fast_ma, self.slow_ma):
            if not self.position: 
                self.buy()
        elif crossover(self.slow_ma, self.fast_ma):
            self.position.close()  

### 2. Relative Strength Index (RSI)

Buys when the RSI shoots up above 30. Sells when the RSI drops below 70.

In [219]:
class RSI(Strategy):
    display_name = "RSI"
    timeperiod = 14
    
    def init(self):
        close = self.data.Close
        self.rsi = self.I(talib.RSI, close, timeperiod=self.timeperiod)
    def next(self):
        if self.rsi[-2] < 30 and self.rsi[-1] >= 30 and not self.position:
            self.buy()
        elif self.rsi[-2] > 70 and self.rsi[-1] <= 70 and self.position:
            self.sell()

### 3. Buy and Hold

Buys on the first trading day and then holds for the entire backtest period.

In [220]:
class BuyAndHold(Strategy):
    display_name = "BuyAndHold"
    
    def init(self):
        self.bought = False
    def next(self):
        if not self.bought:
            self.buy()
            self.bought = True

### 4. Momentum Investing

Buys when momentum is positive. Sells when momentum is negative.

In [221]:
class Momentum(Strategy):
    display_name = "Momentum"
    
    def init(self):
        close = self.data.Close
        self.momentum = self.I(talib.MOM, close, timeperiod=20)
        self.mom_sma = self.I(talib.SMA, self.momentum, timeperiod=10)
    
    def next(self):
        if self.momentum[-1] > self.mom_sma[-1] and not self.position:
            self.buy()
        elif self.momentum[-1] < self.mom_sma[-1] and self.position:
            self.sell()

## Main Function

---

### Ask User for a Strategy

Provide user a list of available strategies and ask use which strategies to compare.

In [249]:
strategies = {
    'smacrossover': SMAcrossover,
    'rsi': RSI,
    'buyandhold': BuyAndHold,
    'momentum': Momentum
}

def strategy_choice():
    while True:
        strategy_input = input(f"\nStrategy: ").strip().lower()
        if strategy_input in strategies:
            print(f"{strategies[strategy_input].display_name} chosen.")
            return strategy_input
        else:
            print("Invalid or duplicate strategy. Try again.")

print("\nAvailable strategies:")
for strat in strategies.values():
    print(f" - {strat.display_name}")

strategy_key = strategy_choice()


Available strategies:
 - SMAcrossover
 - RSI
 - BuyAndHold
 - Momentum
SMAcrossover chosen.


### Output Strategy Data

In [261]:
df = yf.download('AAPL', period='2y', auto_adjust=True)
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

[*********************100%***********************]  1 of 1 completed


In [262]:
cash, commission, margin = 10000000, 0.001, 0.1

bt = Backtest(df, strategies[strategy_key], cash=cash, commission=commission, margin=margin)
results = bt.run()
print("\nBacktest Results:")
print(results)


Backtest Results:
Start                     2023-08-08 00:00:00
End                       2025-08-07 00:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                    15.13944
Equity Final [$]                          0.0
Equity Peak [$]                14276100.96645
Commissions [$]                  189087.07576
Return [%]                             -100.0
Buy & Hold Return [%]                24.97785
Return (Ann.) [%]                           0
Volatility (Ann.) [%]       16768270181.97565
CAGR [%]                               -100.0
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Alpha [%]                                 NaN
Beta                                      NaN
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                    -23.5453
Max. Drawdown Duration      602 days 00:00:00
Avg. Drawdown Duration      126 days 00:00:00
# Trades       

c:\Users\natha\anaconda3\envs\venv\lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=113: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\natha\anaconda3\envs\venv\lib\site-packages\backtesting\_stats.py:156: RuntimeWarning: divide by zero encountered in log
  equity_log_returns = np.log(equity[1:] / equity[:-1])


In [263]:
bt.plot()

GridPlot(id='p3321', ...)

In [259]:
num_trades = len(results._trades)
print(num_trades)

1
